In [1]:
#Lets start the spark context
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1717476918623_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [2]:
#Lets load the file into spark
df = spark.read.load("/user/root/bank_repo/part-m-00000", format='csv', header=False, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Lets check the data types of all the attributes
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _c27: integer (nu

In [4]:
#Lets check the number of rows present in the dataframe
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [5]:
#Lets create a custom schema based on the requirement
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

schema = StructType([StructField('year', IntegerType(), True),
                     StructField('month', StringType(), True),
                     StructField('day', IntegerType(), True),
                     StructField('weekday', StringType(), True),
                     StructField('hour', IntegerType(), True),
                     StructField('atm_status', StringType(), True),
                     StructField('atm_id', StringType(), True),
                     StructField('atm_manufacturer', StringType(), True),
                     StructField('atm_location', StringType(), True),
                     StructField('atm_streetname', StringType(), True),
                     StructField('atm_street_number', IntegerType(), True),
                     StructField('atm_zipcode', IntegerType(), True),
                     StructField('atm_lat', DoubleType(), True),
                     StructField('atm_lon', DoubleType(), True),
                     StructField('currency', StringType(), True),
                     StructField('card_type', StringType(), True),
                     StructField('transaction_amount', IntegerType(), True),
                     StructField('service', StringType(), True),
                     StructField('message_code', StringType(), True),
                     StructField('message_text', StringType(), True),
                     StructField('weather_lat', DoubleType(), True),
                     StructField('weather_lon', DoubleType(), True),
                     StructField('weather_city_id', IntegerType(), True),
                     StructField('weather_city_name', StringType(), True),
                     StructField('temp', DoubleType(), True),
                     StructField('pressure', IntegerType(), True),
                     StructField('humidity', IntegerType(), True),
                     StructField('wind_speed', IntegerType(), True),
                     StructField('wind_deg', IntegerType(), True),
                     StructField('rain_3h', DoubleType(), True),
                     StructField('clouds_all', IntegerType(), True),
                     StructField('weather_id', IntegerType(), True),
                     StructField('weather_main', StringType(), True),
                     StructField('weather_description', StringType(), True)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#By using the above custom schema, lets load the file again
df = spark.read.load("/user/root/bank_repo/part-m-00000", format='csv', header=False, schema=schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#Lets check the data types of all the attributes
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

In [8]:
#Lets check the number of rows present in the dataframe
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

## Let's create the dimension and fact tables 

### 1. We will now create the location dimension

In [9]:
# Let's create a dataFrame - loc_dim from the main dataframe for which we will pick specific columns and to make sure 
# unique values come, we will use distinct keyword
loc_dim = df.select('atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon').distinct()
loc_dim.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [10]:
# Lets load some libraries which will help in performing window operatings and concatenation, etc.
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Lets create a primary key called "location_id"
DIM_LOCATION = loc_dim.select(row_number().over(Window.orderBy(loc_dim['atm_location'])).alias("location_id"),"*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Lets see few rows
DIM_LOCATION.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
|location_id|        atm_location|atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|  ÃƒËœstergade|                6|       9440| 57.162|   9.73|
|          2|      Aalborg Hallen|  Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|      Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|      Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|      Hobrovej|              440|       9200| 57.005|  9.881|
+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
only showing top 5 rows

In [13]:
# Lets rename few columns as per requirement
DIM_LOCATION = DIM_LOCATION.withColumnRenamed("atm_location", "location") \
                           .withColumnRenamed("atm_streetname", "streetname") \
                           .withColumnRenamed("atm_street_number", "street_number") \
                           .withColumnRenamed("atm_zipcode", "zipcode") \
                           .withColumnRenamed("atm_lat", "lat") \
                           .withColumnRenamed("atm_lon", "lon")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Lets check the schema
DIM_LOCATION.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- location_id: integer (nullable = false)
 |-- location: string (nullable = true)
 |-- streetname: string (nullable = true)
 |-- street_number: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)

In [15]:
# Lets check the count of rows
DIM_LOCATION.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [16]:
# Lets see few rows by using show function
DIM_LOCATION.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------+-------------+-------+------+-----+
|location_id|            location|  streetname|street_number|zipcode|   lat|  lon|
+-----------+--------------------+------------+-------------+-------+------+-----+
|          1|             Aabybro|ÃƒËœstergade|            6|   9440|57.162| 9.73|
|          2|      Aalborg Hallen|Europa Plads|            4|   9000|57.044|9.913|
|          3|Aalborg Storcente...|    Hobrovej|          452|   9200|57.005|9.876|
|          4|Aalborg Storcente...|    Hobrovej|          452|   9200|57.005|9.876|
|          5|         Aalborg Syd|    Hobrovej|          440|   9200|57.005|9.881|
+-----------+--------------------+------------+-------------+-------+------+-----+
only showing top 5 rows

### 2. We will now create the ATM dimension

In [17]:
# Let's create a dataFrame - atm_dim from the main dataframe for which we will pick specific columns and to make sure 
# unique values come, we will use distinct keyword
atm_dim = df.select("atm_id", "atm_manufacturer", "atm_location", "atm_streetname", "atm_street_number",
                    "atm_zipcode", "atm_lat", "atm_lon").distinct()
atm_dim.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [18]:
# Lets rename few columns as per requirement
atm_dim = atm_dim.withColumnRenamed("atm_lat", "lat") \
                 .withColumnRenamed("atm_lon", "lon") \
                 .withColumnRenamed("atm_zipcode", "zipcode") \
                 .withColumnRenamed("atm_street_number", "street_number") \
                 .withColumnRenamed("atm_streetname", "streetname") \
                 .withColumnRenamed("atm_location", "location")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Let's join atm_dim with DIM_LOCATION using left join
DIM_ATM = atm_dim.join(DIM_LOCATION, on=['lat','lon', 'zipcode', 'street_number', 'streetname', 'location'], how="left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Lets see few rows by using show function
DIM_ATM.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-------+-------------+---------------+--------------------+------+----------------+-----------+
|   lat|   lon|zipcode|street_number|     streetname|            location|atm_id|atm_manufacturer|location_id|
+------+------+-------+-------------+---------------+--------------------+------+----------------+-----------+
|55.398|11.342|   4200|           29|Mariendals Alle|            Slagelse|    31|             NCR|         88|
|57.441|10.537|   9900|           48|   Danmarksgade|       Frederikshavn|    40| Diebold Nixdorf|         27|
|55.505| 9.457|   6000|          135|       Vejlevej|             Kolding|    19|             NCR|         62|
|57.005| 9.876|   9200|          452|       Hobrovej|Aalborg Storcente...|    87|             NCR|          4|
|56.795|  8.86|   7900|            1|    Kirketorvet|NykÃƒÂ¸bing Mors ...|    59| Diebold Nixdorf|         73|
+------+------+-------+-------------+---------------+--------------------+------+----------------+-----------+
o

In [21]:
# Lets rename few columns as per requirement
DIM_ATM = DIM_ATM.withColumnRenamed("atm_id", "atm_number")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Lets see few rows by using show function
DIM_ATM.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-------+-------------+---------------+--------------------+----------+----------------+-----------+
|   lat|   lon|zipcode|street_number|     streetname|            location|atm_number|atm_manufacturer|location_id|
+------+------+-------+-------------+---------------+--------------------+----------+----------------+-----------+
|55.398|11.342|   4200|           29|Mariendals Alle|            Slagelse|        31|             NCR|         88|
|57.441|10.537|   9900|           48|   Danmarksgade|       Frederikshavn|        40| Diebold Nixdorf|         27|
|55.505| 9.457|   6000|          135|       Vejlevej|             Kolding|        19|             NCR|         62|
|57.005| 9.876|   9200|          452|       Hobrovej|Aalborg Storcente...|        87|             NCR|          4|
|56.795|  8.86|   7900|            1|    Kirketorvet|NykÃƒÂ¸bing Mors ...|        59| Diebold Nixdorf|         73|
+------+------+-------+-------------+---------------+--------------------+------

In [23]:
# We will pick the columns which are needed as per requirement by using select
DIM_ATM = DIM_ATM.select("atm_number", "atm_manufacturer", "location_id").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Lets see few rows by using show function
DIM_ATM.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+-----------+
|atm_number|atm_manufacturer|location_id|
+----------+----------------+-----------+
|         8|             NCR|         30|
|        97|             NCR|         51|
|       113| Diebold Nixdorf|         89|
|        51|             NCR|         29|
|        64|             NCR|         43|
+----------+----------------+-----------+
only showing top 5 rows

In [25]:
# Lets rename few rows as per requirement
DIM_ATM = DIM_ATM.withColumnRenamed("location_id", "atm_location_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Lets see few rows by using show function
DIM_ATM.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+---------------+
|atm_number|atm_manufacturer|atm_location_id|
+----------+----------------+---------------+
|         8|             NCR|             30|
|        97|             NCR|             51|
|       113| Diebold Nixdorf|             89|
|        51|             NCR|             29|
|        64|             NCR|             43|
+----------+----------------+---------------+
only showing top 5 rows

In [27]:
# Lets create a primary key called "atm_id"
DIM_ATM = DIM_ATM.select(row_number().over(Window.orderBy(DIM_ATM['atm_number'])).alias("atm_id"), "*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Lets see few rows by using show function
DIM_ATM.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+---------------+
|atm_id|atm_number|atm_manufacturer|atm_location_id|
+------+----------+----------------+---------------+
|     1|         1|             NCR|             74|
|     2|        10|             NCR|             76|
|     3|       100|             NCR|             56|
|     4|       101|             NCR|             17|
|     5|       102|             NCR|              3|
+------+----------+----------------+---------------+
only showing top 5 rows

In [29]:
# Lets check the count of rows
DIM_ATM.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

### 3. We will now create the Date dimension

In [30]:
# Let's create a dataFrame - DIM_DATE from the main dataframe for which we will pick specific columns and to make sure 
# unique values come, we will use distinct keyword'
DIM_DATE = df.select('year', 'month', 'day', 'hour', 'weekday')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Lets see few rows by using show function
DIM_DATE.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+-------+
|year|  month|day|hour|weekday|
+----+-------+---+----+-------+
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
+----+-------+---+----+-------+
only showing top 5 rows

In [32]:
# Lets import few necessary libraries for building a new column called full_date_time and converting it to timestamp 
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit
DIM_DATE = DIM_DATE.withColumn('full_date_time', to_timestamp(concat(DIM_DATE.year, lit('-'), DIM_DATE.month,
                                 lit('-'), DIM_DATE.day, lit(' '), DIM_DATE.hour), 'yyyy-MMMM-d H'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Lets see few rows by using show function
DIM_DATE.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+-------+-------------------+
|year|  month|day|hour|weekday|     full_date_time|
+----+-------+---+----+-------+-------------------+
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
+----+-------+---+----+-------+-------------------+
only showing top 5 rows

In [34]:
# We will pick the columns which are needed as per requirement by using select
DIM_DATE = DIM_DATE.select('full_date_time', 'year', 'month', 'day', 'hour', 'weekday').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Lets create a primary key called "date_id"
DIM_DATE = DIM_DATE.select(row_number().over(Window.orderBy(DIM_DATE['full_date_time'])).alias('date_id'), "*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Lets see few rows by using show function
DIM_DATE.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-------+---+----+-------+
|date_id|     full_date_time|year|  month|day|hour|weekday|
+-------+-------------------+----+-------+---+----+-------+
|      1|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|
|      2|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|
|      3|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|
|      4|2017-01-01 03:00:00|2017|January|  1|   3| Sunday|
|      5|2017-01-01 04:00:00|2017|January|  1|   4| Sunday|
+-------+-------------------+----+-------+---+----+-------+
only showing top 5 rows

In [37]:
# Lets check the count of rows
DIM_DATE.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

### 4. We will now create the Card type dimension

In [38]:
# Let's create a dataFrame - card_type_dim from the main dataframe for which we will pick specific columns and to make sure 
# unique values come, we will use distinct keyword'
card_type_dim = df.select('card_type').distinct()
card_type_dim.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [39]:
# Lets see few rows by using show function
card_type_dim.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           card_type|
+--------------------+
|Visa Dankort - on-us|
|  Mastercard - on-us|
|         HÃƒÂ¦vekort|
|            VisaPlus|
|     Dankort - on-us|
+--------------------+
only showing top 5 rows

In [40]:
# Lets create a primary key called "card_type_id"
DIM_CARD_TYPE = card_type_dim.select(row_number().over(Window.orderBy(card_type_dim['card_type'])).alias('card_type_id'), "*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Lets see few rows by using show function
DIM_CARD_TYPE.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------------+
|card_type_id|          card_type|
+------------+-------------------+
|           1|             CIRRUS|
|           2|            Dankort|
|           3|    Dankort - on-us|
|           4|        HÃƒÂ¦vekort|
|           5|HÃƒÂ¦vekort - on-us|
+------------+-------------------+
only showing top 5 rows

In [42]:
# Lets see the count of rows
DIM_CARD_TYPE.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

### 5. We will now create the fact_atm_trans fact table

In [43]:
#Lets rename few columns
location_fact = df.withColumnRenamed('atm_location', 'location') \
                   .withColumnRenamed('atm_streetname', 'streetname') \
                   .withColumnRenamed('atm_street_number', 'street_number') \
                   .withColumnRenamed('atm_zipcode', 'zipcode') \
                   .withColumnRenamed('atm_lat', 'lat') \
                   .withColumnRenamed('atm_lon', 'lon')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# Lets join above dataframe with DIM_LOCATION using left join
location_fact = location_fact.join(DIM_LOCATION, on=['location', 'streetname', 'street_number', 'zipcode',
                                                      'lat', 'lon'], how="left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# Lets check the count of rows by using count function
location_fact.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [46]:
# Lets check the column names
location_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_id', 'atm_manufacturer', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'location_id']

In [47]:
#Lets rename few columns
location_fact = location_fact.withColumnRenamed('atm_id', 'atm_number') \
                             .withColumnRenamed('location_id', 'atm_location_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# Lets check the column names
location_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_number', 'atm_manufacturer', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_location_id']

In [49]:
# Let's join location_fact with DIM_ATM using left join
atm_fact = location_fact.join(DIM_ATM, on=['atm_number', 'atm_manufacturer', 'atm_location_id'], how="left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# Lets check the column names
atm_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_number', 'atm_manufacturer', 'atm_location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id']

In [51]:
#Lets rename few columns
atm_fact = atm_fact.withColumnRenamed('atm_location_id', 'weather_loc_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Lets check the column names
atm_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id']

In [53]:
# Let's join atm_fact with DIM_DATE using left join
date_fact = atm_fact.join(DIM_DATE, on=['year', 'month', 'day', 'hour', 'weekday'], how="left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# Lets check the column names
date_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time']

In [55]:
# Let's join date_fact with DIM_CARD_TYPE using left join
cardtype_fact = date_fact.join(DIM_CARD_TYPE, on=['card_type'], how="left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
# Lets check the column names
cardtype_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id']

In [57]:
# We will pick the columns which are needed as per requirement by using select
cardtype_fact = cardtype_fact.select(row_number().over(Window.orderBy(cardtype_fact["date_id"])).alias("trans_id"), "*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
# Lets check the column names
cardtype_fact.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['trans_id', 'card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id']

In [59]:
# We will pick the columns which are needed as per requirement by using select
FACT_ATM_TRANS = cardtype_fact.select("trans_id", "atm_id", "weather_loc_id", "date_id", "card_type_id", "atm_status",
                                      "currency", "service", "transaction_amount", "message_code", "message_text", 
                                      "rain_3h", "clouds_all", "weather_id", "weather_main", "weather_description")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
# Lets check the column names
FACT_ATM_TRANS.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['trans_id', 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']

In [61]:
# Lets check the count of rows by using count function
FACT_ATM_TRANS.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

### Lets load all the 4 dimension and fact tables into S3

In [62]:
#Lets write DIM_LOCATION to a S3 bucket
DIM_LOCATION.write.format('csv').option('header','false').save('s3://etlpipelineproject1/sparnod_bank/LocationFolder/'
                                                               ,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
#Lets write DIM_ATM to a S3 bucket
DIM_ATM.write.format('csv').option('header','false').save('s3://etlpipelineproject1/sparnod_bank/ATMFolder/'
                                                          ,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
#Lets write DIM_DATE to a S3 bucket
DIM_DATE.write.format('csv').option('header','false').save('s3://etlpipelineproject1/sparnod_bank/DateFolder/'
                                                           ,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
#Lets write DIM_CARD_TYPE to a S3 bucket
DIM_CARD_TYPE.write.format('csv').option('header','false').save('s3://etlpipelineproject1/sparnod_bank/CardTypeFolder/'
                                                                ,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
#Lets write FACT_ATM_TRANS to a S3 bucket
FACT_ATM_TRANS.write.format('csv').option('header','false').save('s3://etlpipelineproject1/sparnod_bank/FactATMFolder/'
                                                                 ,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…